This example shows, how to formulate and solve optimal control problem (OCP) with evolutionary algorithms using packages EvoOptControl.jland Evolutionary.jl

The general OCP can be formulated as follows:
$$\min_{x(t), u(t)} \phi(t_0, x(t_0), t_f, x(t_f)) + \int_{t_0}^{t_f}L(x(t), u(t), t) \mathrm{d}t$$
subject to $$\dot{x} = f(x(t), u(t), t)$$ $$c_{\mathrm{min}} < c(x(t), u(t), t) < c_{\mathrm{max}}$$ $$b_{\mathrm{min}} < b(t_0, x(t_0), t_f, x(t_f)) < b_{\mathrm{max}}$$

where $c(x(t), u(t), t)$ represents constraints and $b(t_0, x(t_0), t_f, x(t_f))$ represents boundary conditions. The first equation is general state equation that must be satisfied for given dynamical system.

This problem is quite hard to solve analytically, therefore there are developed many numerical methods for solving this problem. The two big categories are *indirect* methods and *direct* methods. This example is focused on *direct* methods, which can be separated into *single shooting* method, *multiple-shooting* method and *direct collocation* (using h-scheme, hp-scheme, p-scheme - more on those in presentation), which can transform original problem into nonlinear program (NLP).

Generally the NLP can be intractable to solve (can be heavily nonconvex). Therefore I'm presenting a package, that will transform the original problem into NLP and try to solve it with evolutionary algorithms.

Lets start with a relatively simple example - trajectory optimization for single pendulum. The cost that will be minimized will be the energy of control input on interval (0, 1).

$$\min_{x(t), u(t)} \int_0^1 u^2(t)\mathrm{d}t$$

Lets say we want to bring the pendulum from stable position ($\theta = 0$, $\omega = 0$) to an unstable position ($\theta = \pi$, $\omega = 0$) in the time interval with minimum energy. The pendulum dynamics can be written as

$$\dot{\theta} = \omega$$ $$\dot{\omega} = -\frac{g}{l}\sin(\theta) + u$$

But we must state that the state and control is bounded $\theta \in [-2\pi, 2\pi]$, $\omega \in [-10, 10]$ and $u \in [-1, 1]$

Lets model this example, with $l = 1$

In [4]:
using EvoOptControl
using JSON

# Pendulum ODE
function pendulum_dynamics(x, u, t)
    θ, ω = x
    dx = [ω; -9.81/1 * sin(θ) + u[1]]
    return dx
end

# Running cost (term in the integral part of the cost)
running_cost(t, x, u) = u[1]^2

# Terminal cost
terminal_cost(t, x) = 0

# Time span on which is OCP solved
tspan = (0.0, 1.0)

# Boundary conditions
x0 = [0.0; 0.0]
xf = [pi; 0.0]

# Utility variables
state_dim = 2 # dimension of state
control_dim = 1 # dimension of control

# State & control constraints
state_constraints = [StateConstraint(identity, [-2pi, -10.0], [2pi, 10.0])]
control_constraints = [ControlConstraint(identity, [-1.0], [1.0])]

1-element Vector{ControlConstraint}:
 ControlConstraint(identity, [-5.0], [5.0])

Lets now define the direct collocation method. As a now, only 2 methods from h-scheme are implemented: `BackwardEulerCollocation` and `TrapezoidalCollocation`. Few of hp-scheme are developed. Each collocation method takes number of intervals `N` as an argument. The higher the `N` is, the harder the NLP is but the transcription (transformation into NLP) is more accurate. Lets create 3 instances of `BackwardEulerCollocation` and `TrapezoidalCollocation` with different `N`.

In [14]:
N_small = 10
N_medium = 50
N_high = 200

# Backwar Euler method
method1_small = BackwardEulerCollocation(N_small)
method1_medium = BackwardEulerCollocation(N_medium)
method1_high = BackwardEulerCollocation(N_high)

# Trapezoidal method
method2_small = TrapezoidalCollocation(N_small)
method2_medium = TrapezoidalCollocation(N_medium)
method2_high = TrapezoidalCollocation(N_high)

TrapezoidalCollocation(200)

Now it is time do define the OCP.

In [6]:
prob1_small = OCProblem(running_cost, terminal_cost, tspan, pendulum_dynamics, x0, xf, state_dim, control_dim, method1_small, state_constraints, control_constraints)
prob1_medium = OCProblem(running_cost, terminal_cost, tspan, pendulum_dynamics, x0, xf, state_dim, control_dim, method1_medium, state_constraints, control_constraints)
prob1_high = OCProblem(running_cost, terminal_cost, tspan, pendulum_dynamics, x0, xf, state_dim, control_dim, method1_high, state_constraints, control_constraints)

prob2_small = OCProblem(running_cost, terminal_cost, tspan, pendulum_dynamics, x0, xf, state_dim, control_dim, method2_small, state_constraints, control_constraints)
prob2_medium = OCProblem(running_cost, terminal_cost, tspan, pendulum_dynamics, x0, xf, state_dim, control_dim, method2_medium, state_constraints, control_constraints)
prob2_high = OCProblem(running_cost, terminal_cost, tspan, pendulum_dynamics, x0, xf, state_dim, control_dim, method2_high, state_constraints, control_constraints)


OCProblem{TrapezoidalCollocation}(running_cost, terminal_cost, (0.0, 1.0), pendulum_dynamics, [0.0, 0.0], [3.141592653589793, 0.0], 2, 1, TrapezoidalCollocation(200), StateConstraint[StateConstraint(identity, [-6.283185307179586, -10.0], [6.283185307179586, 10.0])], ControlConstraint[ControlConstraint(identity, [-5.0], [5.0])])

Lets now discuss, which EA will be used. I chose 2 EAs, the first one being CMA-ES with parameters $\mu = 100$ and $\lambda = 200$. The second EA will be $(\mu/\rho , \lambda)$ ES with $\mu = 100$, $\rho = 40$ and $\lambda=200$ both with population size of 300 each ran for 20 trials with 150 iterations for each run.

Crossover operator for ES is going to be averaging, mutation is going to be isotropic gaussian (one $\sigma$ for population)

In [15]:
using Evolutionary

N_runs = 20
population_size = 300
iterations = 150

# CMA-ES
ea_method1 = CMAES(mu = 100, lambda = 200)

# (mu/rho + lambda) ES
ea_method2_small = ES(initStrategy = IsotropicStrategy(N_small*(state_dim + control_dim)), srecombination = average, recombination = average, mutation = gaussian,
                   smutation = gaussian, mu = 100, rho = 40, lambda = 200, selection = :comma)
    
ea_method2_medium = ES(initStrategy = IsotropicStrategy(N_medium*(state_dim + control_dim)), srecombination = average, recombination = average, mutation = gaussian,
                    smutation = gaussian, mu = 100, rho = 40, lambda = 200, selection = :comma)
        
ea_method2_high = ES(initStrategy = IsotropicStrategy(N_high*(state_dim + control_dim)), srecombination = average, recombination = average, mutation = gaussian,
                  smutation = gaussian, mu = 100, rho = 40, lambda = 200, selection = :comma)

(100/40+200)-ES

Lets prepare storage for the optimization and lets optimize!

In [ ]:
# Storage for all optimization data
optimization_results = Dict{String, Dict{String, Dict{String, Vector{OCPSolution}}}}()

# Storage for particular case
backward_small_cmaes = Vector{OCPSolution}(undef, N_runs)
backward_small_es = Vector{OCPSolution}(undef, N_runs)
backward_medium_cmaes = Vector{OCPSolution}(undef, N_runs)
backward_medium_es = Vector{OCPSolution}(undef, N_runs)
backward_high_cmaes = Vector{OCPSolution}(undef, N_runs)
backward_high_es = Vector{OCPSolution}(undef, N_runs)

trapezoidal_small_cmaes = Vector{OCPSolution}(undef, N_runs)
trapezoidal_small_es = Vector{OCPSolution}(undef, N_runs)
trapezoidal_medium_cmaes = Vector{OCPSolution}(undef, N_runs)
trapezoidal_medium_es = Vector{OCPSolution}(undef, N_runs)
trapezoidal_high_cmaes = Vector{OCPSolution}(undef, N_runs)
trapezoidal_high_es = Vector{OCPSolution}(undef, N_runs)

# Main loop
for i = 1:N_runs
    sol_bw_small_cmaes = solve_ocp(prob1_small, ea_method1, population_size, iterations)
    sol_bw_small_es = solve_ocp(prob1_small, ea_method2_small, population_size, iterations)
    sol_bw_medium_cmaes = solve_ocp(prob1_medium, ea_method1, population_size, iterations)
    sol_bw_medium_es = solve_ocp(prob1_medium, ea_method2_medium, population_size, iterations)
    sol_bw_high_cmaes = solve_ocp(prob1_high, ea_method1, population_size, iterations)
    sol_bw_high_es = solve_ocp(prob1_high, ea_method2_high, population_size, iterations)

    sol_tr_small_cmaes = solve_ocp(prob2_small, ea_method1, population_size, iterations)
    sol_tr_small_es = solve_ocp(prob2_small, ea_method2_small, population_size, iterations)
    sol_tr_medium_cmaes = solve_ocp(prob2_medium, ea_method1, population_size, iterations)
    sol_tr_medium_es = solve_ocp(prob2_medium, ea_method2_medium, population_size, iterations)
    sol_tr_high_cmaes = solve_ocp(prob2_high, ea_method1, population_size, iterations)
    sol_tr_high_es = solve_ocp(prob2_high, ea_method2_high, population_size, iterations)

    backward_small_cmaes[i] = create_OCPSolution(prob1_small, sol_bw_small_cmaes)
    backward_small_es[i] = create_OCPSolution(prob1_small, sol_bw_small_es)
    backward_medium_cmaes[i] = create_OCPSolution(prob1_medium, sol_bw_medium_cmaes)
    backward_medium_es[i] = create_OCPSolution(prob1_medium, sol_bw_medium_es)
    backward_high_cmaes[i] = create_OCPSolution(prob1_high, sol_bw_high_cmaes)
    backward_high_es[i] = create_OCPSolution(prob1_high, sol_bw_high_es)

    trapezoidal_small_cmaes[i] = create_OCPSolution(prob2_small, sol_tr_small_cmaes)
    trapezoidal_small_es[i] = create_OCPSolution(prob2_small, sol_tr_small_es)
    trapezoidal_medium_cmaes[i] = create_OCPSolution(prob2_medium, sol_tr_medium_cmaes)
    trapezoidal_medium_es[i] = create_OCPSolution(prob2_medium, sol_tr_medium_es)
    trapezoidal_high_cmaes[i] = create_OCPSolution(prob2_high, sol_tr_high_cmaes)
    trapezoidal_high_es[i] = create_OCPSolution(prob2_high, sol_tr_high_es)
end

optimization_results["BackwardEuler"] = Dict("CMAES" => Dict("10" => backward_small_cmaes, "50" => backward_medium_cmaes, "200"=>backward_high_cmaes), "ES" => Dict("10" => backward_small_es, "50" => backward_medium_es, "200" => backward_high_es))
optimization_results["Trapezoidal"] = Dict("CMAES" => Dict("10" => trapezoidal_small_cmaes, "50" => trapezoidal_medium_cmaes, "200"=>trapezoidal_high_cmaes), "ES" => Dict("10" => trapezoidal_small_es, "50" => trapezoidal_medium_es, "200" => trapezoidal_high_es))

# Possible save into json, not needed for now -> already saved and saved 80 mins of suffering
open("opt_results.json", "w") do f
    JSON.print(f, optimization_results)
end

Iter     Function value
     0              Inf
 * time: 0.03900003433227539
     1   23.88846665590513
 * time: 0.377000093460083
     2   20.37865480300634
 * time: 0.38900017738342285
     3   18.228508995080237
 * time: 0.40400004386901855
     4   17.369768892849724
 * time: 0.4200000762939453
     5   18.02338447934975
 * time: 0.440000057220459
     6   17.101726394950926
 * time: 0.45800018310546875
     7   16.3000043347794
 * time: 0.5190000534057617
     8   16.517342607819035
 * time: 0.5380001068115234
     9   15.690783026962281
 * time: 0.5500001907348633
    10   15.175970467943449
 * time: 0.5690000057220459
    11   15.063404530375642
 * time: 0.5900001525878906
    12   14.863435103754881
 * time: 0.6030001640319824
    13   14.931482837154334
 * time: 0.6470000743865967
    14   14.826337569704812
 * time: 0.7720000743865967
    15   14.498433246834564
 * time: 0.7899999618530273
    16   14.522058849291358
 * time: 0.8230001926422119
    17   14.346148995272934
 * 